In [ ]:
import inflect
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.datasets import WikiText2

import torch.nn.functional as F
from tqdm import tqdm
import spacy
import numpy as np
import random







[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data = "/content/drive/My Drive/NLP spam data/Embedings/tiny.txt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open(data, 'r') as f:
  txt_open = f.read()


# print(len(txt_open))

def preprocess(text, stopword_remove = 0):
  p = inflect.engine()

  # lowercasting
  text = text.lower()

  # character removal
  chars_to_remove = '.<.*?>|[\[\]!@#$%^&*()_+{}|:"<>?`~\-=\';\.,]|[^\w\s]'
  chars_to_remove = [i for i in chars_to_remove]

  # tokenizing
  tokenizing = [i for i in text.split()]
  tokenizing2 = [i for i in tokenizing if i not in chars_to_remove]


  # digit to text
  digit_to_word = [p.number_to_words(word) if word.isdigit() else word for word in tokenizing2]


  # stopword removal
  if stopword_remove:
    stops = set(stopwords.words("english"))
    filtered_tokens = [token for token in digit_to_word if token not in stops]
  else:
    filtered_tokens = digit_to_word

  # lemmitize
  l = WordNetLemmatizer()
  lemmatized_tokens = [l.lemmatize(token) for token in filtered_tokens]


  return lemmatized_tokens

preprocessed = preprocess(txt_open)
len(txt_open)
# len(preprocessed)

4535

In [ ]:
# executable implementation of your neural network. E
ciency of your implementation is a
# plus
#  Calculation of word embeddings with your model on at least one of the two given datasets
#  Visualization of Word Embeddings with methods like t-SNE. For this you may use opensource software.

/usr/local/lib/python3.10/dist-packages/IPython/core/inputtransformer2.py:478: UserWarning: `make_tokens_by_line` received a list of lines which do not have lineending markers ('\n', '\r', '\r\n', '\x0b', '\x0c'), behavior will be unspecified
  warnings.warn("`make_tokens_by_line` received a list of lines which do not have lineending markers ('\\n', '\\r', '\\r\\n', '\\x0b', '\\x0c'), behavior will be unspecified")


In [ ]:
def create_dataset(file_path, preprocess):
  with open(file_path, 'r') as f:
    txt_open = f.read()

  tokenized = preprocess(txt_open)

  vocab = set(tokenized)

  # word to index and index to word mapping

  word_to_index = {word: i for i, word in enumerate(vocab)}
  index_to_word = {i: word for i, word in enumerate(vocab)}

  # generate training data

  data = []

  for i in range(2, len(tokenized) - 2):
    context = [
        tokenized[i - 2],
        tokenized[i - 1],
        tokenized[i + 1],
        tokenized[i + 2]
    ]
    target = tokenized[i]

    context_index = [word_to_index[w] for w in context]
    target_index = word_to_index[target]

    data.append((context_index, target_index))

# , word_to_index, index_to_word
  return [data, word_to_index, index_to_word]

data = create_dataset(data, preprocess = preprocess)

data[1]



{'it': 0,
 'experiment': 1,
 'chilly': 2,
 'two thousand and thirty-six': 3,
 'northern': 4,
 'bahrain': 5,
 'book': 6,
 'alamo': 7,
 'novel': 8,
 'waldorf': 9,
 'carter': 10,
 'show': 11,
 'devil': 12,
 'music': 13,
 'song': 14,
 'use': 15,
 'on': 16,
 'definitely': 17,
 'one thousand, nine hundred and fifty-eight': 18,
 'brasserie': 19,
 'black': 20,
 'call': 21,
 'hiptronix': 22,
 'negra': 23,
 'best': 24,
 'movement': 25,
 'series': 26,
 'when': 27,
 'deal50': 28,
 'wood': 29,
 'warmer': 30,
 'sylvia': 31,
 'am': 32,
 'name': 33,
 'eight': 34,
 'indonesian': 35,
 'coffee': 36,
 'now': 37,
 'newest': 38,
 'ashita': 39,
 'rain': 40,
 'i': 41,
 'kurjochin': 42,
 'dimensional': 43,
 'for': 44,
 'point': 45,
 'babar': 46,
 'pandora': 47,
 'zen': 48,
 'jam': 49,
 'showing': 50,
 'can': 51,
 'shark': 52,
 'mann': 53,
 'titled': 54,
 'new': 55,
 'nelson': 56,
 '25,': 57,
 'hole': 58,
 'netflix': 59,
 'mariana': 60,
 'party': 61,
 'orion': 62,
 'four': 63,
 "i'm": 64,
 'park': 65,
 'wrestle

In [ ]:
# embeddings layer

class CustomEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CustomEmbedding, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.weight = nn.Parameter(torch.randn(vocab_size, embedding_dim))

    def forward(self, inputs):
        embedded = self.weight[inputs]
        return embedded


class CBOW(nn.Module):
    def __init__(self, vocab_size = 4, embedding_size = 3, window_size = 2):
      super().__init__()
      self.embeddings = CustomEmbedding(vocab_size, embedding_size)
      self.linear = nn.Linear(embedding_size, vocab_size)
      self.window_size = window_size

    def forward(self, inputs):
      output = self.embeddings(inputs).mean(1).squeeze(1) # there is also a sum approach
      out = self.linear(output)
      log_probs = F.log_softmax(out, dim=1)
      return log_probs


data, word_to_index, index_to_word = data

embedding_size = 100
vocab_size = len(word_to_index)
# loss_function = nn.CrossEntropyLoss()
loss_function = nn.NLLLoss()


model = CBOW(embedding_size=embedding_size, vocab_size=vocab_size)
optimizer = optim.Adam(model.parameters(), lr = 0.005)


context_data = torch.tensor([ex[0] for ex in data])
labels = torch.tensor([ex[1] for ex in data])

# creating  a dataset

dataset = torch.utils.data.TensorDataset(context_data, labels)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle = True)


# for i in dataset:
#   print(i)



In [ ]:
# train loop
num_epochs = 1000

total = 0
final_loss = 0
for epoch in tqdm(range(num_epochs)):
  for context, label in dataloader:
    output = model(context)
    loss = loss_function(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total += loss.item()

    final_loss = loss.item()
    print(loss.item())


# print()
# print("total: ", total)
# print("final: ", final_loss)


  0%|          | 3/1000 [00:00<00:44, 22.37it/s]

5.388812132878229e-05
9.513655095361173e-05
7.076498150127009e-05
6.856772961327806e-05
8.816459740046412e-05
0.029963139444589615
7.328450010390952e-05
6.853076774859801e-05
0.0001714014506433159
8.851395250530913e-05
6.365154695231467e-05
8.031067409319803e-05
0.026238176971673965
7.723105954937637e-05
0.00010306302283424884
0.021710557863116264
8.398293721256778e-05
0.030793046578764915
7.23750563338399e-05
0.00011993849329883233
0.00010305979230906814
9.287108696298674e-05
0.00010512542212381959
8.780456118984148e-05
0.00010091066360473633
0.0001325115154031664
8.814904867904261e-05
0.0001237086544279009
5.33220918441657e-05
0.0001012823122437112
7.530661241617054e-05
6.624209345318377e-05
6.889900396345183e-05
7.649457984371111e-05
0.0001379635214107111
7.81929338700138e-05
8.560592686990276e-05
5.687950033461675e-05
7.289598579518497e-05
0.00010138779180124402
6.932031101314351e-05
0.00012014356616418809
9.406930621480569e-05
0.0001473575975978747
0.029741860926151276
7.505730900

  1%|          | 9/1000 [00:00<00:43, 22.63it/s]

8.913274359656498e-05
0.03136011213064194
8.10640340205282e-05
0.00015458073175977916
7.285918400157243e-05
0.00013197449152357876
6.235839100554585e-05
6.035116530256346e-05
9.064214100362733e-05
0.000112153087684419
7.512375304941088e-05
0.00010920783824985847
0.00010695061064325273
8.154750685207546e-05
8.035876089707017e-05
9.141099144471809e-05
9.705586126074195e-05
6.288025906542316e-05
0.00011088155588367954
7.232421194203198e-05
0.0177646242082119
6.363545981002972e-05
6.315586506389081e-05
6.476877024397254e-05
0.0001065553879016079
0.032794222235679626
7.120911322999746e-05
6.587401730939746e-05
0.035490553826093674
6.223921081982553e-05
8.213973342208192e-05
7.602828554809093e-05
6.0977046814514324e-05
0.00010198170639341697
6.37630291748792e-05
0.02661309391260147
5.9294547099852934e-05
7.44102435419336e-05
9.584746294422075e-05
7.04372941982001e-05
8.229671948356554e-05
8.685639477334917e-05
7.744778849883005e-05
0.00010708503396017477
9.025238250615075e-05
6.5271095081698

  1%|          | 12/1000 [00:00<00:45, 21.72it/s]

...

 19%|█▉        | 189/1000 [00:09<00:37, 21.60it/s]

2.381842932663858e-05
0.0323777049779892
7.189667030615965e-06
5.759229679824784e-06
6.755157755833352e-06
4.205839559290325e-06
4.9285422392131295e-06
4.950892616761848e-06
1.0240349183732178e-05
3.0035000236239284e-05
0.023879924789071083
5.349490947992308e-06
6.198850314831361e-06
1.520080604677787e-05
0.02027018554508686
5.915730980632361e-06
1.282160883420147e-05
7.655312401766423e-06
0.055350203067064285
6.9848524617555086e-06
6.511726041935617e-06
6.359006420098012e-06
5.185572717891773e-06
6.712919002893614e-06
6.958714493521256e-06
3.7923362015135353e-06
1.4743228348379489e-05
4.932265710522188e-06
1.0344596375944093e-05
4.780580275109969e-05
9.715236956253648e-06
9.486936505709309e-06
3.833315076917643e-06
6.809765181969851e-06
5.72944827581523e-06
0.02264736406505108
6.593668786081253e-06
1.6750345821492374e-05
1.8248723790748045e-05
5.040301402914338e-06
9.692876119515859e-06
5.427722499007359e-06
0.023230092599987984
4.764629920828156e-06
5.0142198233515956e-06
4.947170054

 19%|█▉        | 190/1000 [00:09<00:38, 20.90it/s]

4.012128101749113e-06
1.0370710697316099e-05
4.11270866607083e-06
5.9753319874289446e-06
4.928543148707831e-06
8.553068255423568e-06
9.689035323390272e-06
4.608162271324545e-06
1.956261257873848e-05
4.537819040706381e-05
3.911541625711834e-06
4.399554200062994e-06
1.0352344361308496e-05
4.935990091325948e-06
0.02199893817305565
4.753454504680121e-06
1.3768089047516696e-05
5.323417099134531e-06
4.488960712478729e-06
3.993501195509452e-06
0.018655873835086823
0.026307931169867516
4.943443855154328e-06
3.378013934707269e-05
6.99973998052883e-06
0.028664633631706238
1.2148980204074178e-05
9.871698239294346e-06
4.608167728292756e-06
6.846998076071031e-06
4.217014520691009e-06
3.978598670073552e-06
4.66776918983669e-06
4.3809268390759826e-06
1.4274713976192288e-05
6.414889867301099e-06
2.0220893929945305e-05
4.287796400603838e-06
0.024005604907870293
6.817225767008495e-06
4.988139153283555e-06
6.586185918422416e-06
6.999689503572881e-06
8.999937563203275e-06
7.91598176874686e-06


KeyboardInterrupt: ignored

In [ ]:
word_embeddings = model.embeddings.weight.data.numpy()

s = random.sample(preprocess(txt_open), k = 25)

em = ['i', 'like', 'to', 'this', 'out']

for i in s:
  em.append(i)

em = set(em)
len(word_embeddings)

354

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

selected = []

for word in em:
  idx = c_dataset.word_to_idx[word]
  selected.append(word_embeddings[idx])

selected = np.array(selected)

tsne = TSNE(n_components=2, perplexity = 2)
embeddings_2d = tsne.fit_transform(selected)

# Scatter plot of the embeddings
# plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
# plt.xlabel('Dimension 1')
# plt.ylabel('Dimension 2')
# plt.title('t-SNE Visualization of Word Embeddings')
# plt.show()
s = 6
fig, ax = plt.subplots(figsize=(s, s))
ax.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

# Add labels to the scatter plot
for i, word in enumerate(em):
    ax.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=10)

plt.show()

AttributeError: ignored

In [ ]:
  # Calculation of word embeddings with your model on at least one of the two given datasets

np.savetxt('word_embeddings1.tsv', word_embeddings, delimiter='\t', fmt='%.6f')


In [ ]:
# Visualization of Word Embeddings with methods like t-SNE
from google.colab import files

# Save word embeddings as TSV
# np.savetxt('word_embeddings.tsv', word_embeddings, delimiter='\t')

# Download the saved file
# files.download('word_embeddings.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# for presentation

inputs = torch.tensor([[1, 3, 2, 0], [2, 0, 3, 1]])


In [ ]:
weight = [[0.1, 0.2, 0.3],
          [0.4, 0.5, 0.6],
          [0.7, 0.8, 0.9],
          [1.0, 1.1, 1.2]]


In [ ]:
output = [[0.4, 0.5, 0.6], [1.0, 1.1, 1.2], [0.7, 0.8, 0.9], [0.1, 0.2, 0.3]] * W
           = [[0.4*W11, 0.5*W12, 0.6*W13, 0.7*W14],
              [1.0*W21, 1.1*W22, 1.2*W23, 1.3*W24],
              [0.7*W31, 0.8*W32, 0.9*W33, 0.1*W34],
              [0.1*W41, 0.2*W42, 0.3*W43, 0.4*W44]]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class SkipGramDataset(Dataset):
    def __init__(self, corpus, window_size):
        self.corpus = corpus
        self.window_size = window_size
        self.data = self._generate_data()
    def _generate_data(self):
        data = []
        for i in range(len(self.corpus)):
            center_word = self.corpus[i]
            context_words = []
            for j in range(i - self.window_size, i + self.window_size + 1):
                if j != i and 0 <= j < len(self.corpus):
                    context_words.append(self.corpus[j])
            data.append((center_word, context_words))
        return data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        center_word, context_words = self.data[index]
        return center_word, context_words
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, center_word):
        embedded = self.embedding(center_word)
        output = self.linear(embedded)
        return output


corpus = create_dataset(data, preprocess = preprocess)
window_size = 2
vocab_size = len(set(corpus))
embedding_dim = 10
learning_rate = 0.001
batch_size = 4
num_epochs = 10


dataset = SkipGramDataset(corpus, window_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = SkipGram(vocab_size, embedding_dim)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0.0
    for center_word, context_words in dataloader:
        optimizer.zero_grad()
        output = model(center_word)
        loss = loss_func(output.view(-1, vocab_size), context_words.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

